In [2]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier #Supervised
from sklearn.neural_network import MLPClassifier
from sklearn import svm # Supervised 
from sklearn.tree import DecisionTreeClassifier

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from numpy import load, save
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from collections import Counter
import time
from sklearn.metrics import f1_score
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import numpy.linalg as npl
#from decomp_utils import *
#from pca_ad_utils1 import *
from numpy import load,save 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.neighbors import KernelDensity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [5]:
plt.rcParams["figure.figsize"] = (20,10)
from sklearn.metrics import auc


In [6]:
plt.rcParams["figure.figsize"] = (20,10)
import sys 
sys.path.append(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Code')
from pca_ad_utils import *
from decomp_utils import *
from customizedLossSynthetic import *

# I. Data import 

Data used included volatilities, spread, rates ... 

The train and test samples are from years 2018/2019. Since the sliding window technique was used to augment the data a particular attention was brought to avoid having close time series at the same time in the training and test sample which could result in a bias in the final prediction and misevaluation of the model. 

In [7]:
X_trainUnderdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\XTrainUnder.csv').drop(['Unnamed: 0'],axis=1)
y_trainUnderdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\yTrainUnder.csv').drop(['Unnamed: 0'],axis=1)

X_testUnderdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\XTestUnder.csv').drop(['Unnamed: 0'],axis=1)
y_testUnderdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\yTestUnder.csv').drop(['Unnamed: 0'],axis=1)
xtrain = np.array(X_trainUnderdf.iloc[:,:-3])
xtest = np.array(X_testUnderdf.iloc[:,:-3])

y_train = np.array(y_trainUnderdf['NbAnomaly'])
y_test = np.array(y_testUnderdf['NbAnomaly'])
ytrain = np.array([1 if l==1 else 0 for l in y_train])
ytest = np.array([1 if l==1 else 0 for l in y_test])

print(f'Normal observations in train set {list(y_train).count(0)}')
print(f'Outlier observations in train set {list(y_train).count(1)} \n')

print(f'Normal observations in test set {list(y_test).count(0)}')
print(f'Outlier observations in test set {list(y_test).count(1)}')

Normal observations in train set 6010
Outlier observations in train set 6010 

Normal observations in test set 2109
Outlier observations in test set 421


In [8]:
ytrain = [1 if i==1 else 0 for i in ytrain]
ytest = [1 if i==1 else 0 for i in ytest]

In [9]:
print(f'Nb. anomalies in Train :{ytrain.count(1)}')
print(f'Nb. normal in Train :{ytrain.count(0)} \n')

print(f'Nb. anomalies in Test :{ytest.count(1)}')
print(f'Nb. normal in Test :{ytest.count(0)} \n')

Nb. anomalies in Train :6010
Nb. normal in Train :6010 

Nb. anomalies in Test :421
Nb. normal in Test :2109 



In [10]:
ScoresDFTrain = pd.DataFrame(columns=['Algo','Accuracy','Recall','Precision','F1'])
ScoresDFTrain['Algo'] = ['IF','LOF','KNN','DBSCAN','SVM','PCA NN AD']
ScoresDFTest = pd.DataFrame(columns=['Algo','Accuracy','Recall','Precision','F1'])
ScoresDFTest['Algo'] = ['IF','LOF','KNN','DBSCAN','SVM','PCA NN AD']

## II. Testing classic anomaly detection algorithms  

### 1. Isolation forest 

In [13]:
# with balanced training and unbalanced test
clf_if_train = IsolationForest(random_state=0,contamination=0.5)

ypredTrainIF = redefine_if_lof_pred(clf_if_train.fit_predict(xtrain))

s = time.time()
clf_if_test = IsolationForest(random_state=0,contamination=0.16)
ypredTestIF = redefine_if_lof_pred(clf_if_test.fit_predict(xtest))
timeIF  = time.time()-s
print(f'Exe. time :{timeIF}\n')

scoresTrainIF = evalalg(ytrain,ypredTrainIF)
scoresTestIF = evalalg(ytest,ypredTestIF)


print(f'Accuracy on Train : {accuracy_score(ytrain,ypredTrainIF)}')
print(f'Accuracy on Test : {accuracy_score(ytest,ypredTestIF)}\n')

print(f'Recall on Train : {recall_score(ytrain,ypredTrainIF)}')
print(f'Recall on Test : {recall_score(ytest,ypredTestIF)}\n')

print(f'Precsion on Train : {precision_score(ytrain,ypredTrainIF)}')
print(f'Precision on Test : {precision_score(ytest,ypredTestIF,pos_label=1)}\n')

print(f'F1-score on Train : {f1_score(ytrain,ypredTrainIF)}')
print(f'F1-score on Test : {f1_score(ytest,ypredTestIF)}\n')

Exe. time :0.6915252208709717

Accuracy on Train : 0.42312811980033277
Accuracy on Test : 0.6964426877470355

Recall on Train : 0.42312811980033277
Recall on Test : 0.0688836104513064

Precsion on Train : 0.42312811980033277
Precision on Test : 0.07160493827160494

F1-score on Train : 0.42312811980033277
F1-score on Test : 0.0702179176755448



In [14]:
print(pd.DataFrame(ypredTrainIF,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestIF,columns=['target'])['target'].value_counts())

0    6010
1    6010
Name: target, dtype: int64
0    2125
1     405
Name: target, dtype: int64


In [15]:
ScoresDFTrain.iloc[0,1:] =  scoresTrainIF
ScoresDFTest.iloc[0,1:] =  scoresTestIF


### 2. Local Outlier factor 

In [16]:
# with balanced training and unbalanced test


clf_lof_train = LocalOutlierFactor(n_neighbors=2,novelty= False,contamination=0.5)
ypredTrainLOF = redefine_if_lof_pred(clf_lof_train.fit_predict(xtrain))

s = time.time()
clf_lof_test = LocalOutlierFactor(n_neighbors=2,novelty= False)
ypredTestLOF = redefine_if_lof_pred(clf_lof_test.fit_predict(xtest))
timeLOF  = time.time()-s

print(f'Exe. time :{timeLOF}')

print(f'Accuracy on Train : {accuracy_score(ytrain,ypredTrainLOF)}')
print(f'Accuracy on Test : {accuracy_score(ytest,ypredTestLOF)}')


print(f'Recall on Train : {recall_score(ytrain,ypredTrainLOF)}')
print(f'Recall on Test : {recall_score(ytest,ypredTestLOF)}\n')

print(f'Precsion on Train : {precision_score(ytrain,ypredTrainLOF)}')
print(f'Precision on Test : {precision_score(ytest,ypredTestLOF,pos_label=1)}\n')

print(f'F1-score on Train : {f1_score(ytrain,ypredTrainLOF)}')
print(f'F1-score on Test : {f1_score(ytest,ypredTestLOF)}\n')

Exe. time :0.201460599899292
Accuracy on Train : 0.5995008319467554
Accuracy on Test : 0.9
Recall on Train : 0.5995008319467554
Recall on Test : 0.498812351543943

Precsion on Train : 0.5995008319467554
Precision on Test : 0.8333333333333334

F1-score on Train : 0.5995008319467554
F1-score on Test : 0.6240713224368499



In [17]:
clf_lof_train = LocalOutlierFactor(n_neighbors=2,novelty= False,contamination=0.5)
clf_lof_test = LocalOutlierFactor(n_neighbors=2,novelty= False)

ypredTrainLOF = redefine_if_lof_pred(clf_lof_train.fit_predict(xtrain))
ypredTestLOF = redefine_if_lof_pred(clf_lof_test.fit_predict(xtest))


scoresTrainLOF = evalalg(ytrain,ypredTrainLOF)
scoresTestLOF = evalalg(ytest,ypredTestLOF)

In [18]:
print(scoresTrainLOF)
print(scoresTestLOF)


(0.5995008319467554, 0.5995008319467554, 0.5995008319467554, 0.5995008319467554)
(0.9, 0.498812351543943, 0.8333333333333334, 0.6240713224368499)


In [19]:
print(pd.DataFrame(ypredTrainLOF,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestLOF,columns=['target'])['target'].value_counts())

1    6010
0    6010
Name: target, dtype: int64
0    2278
1     252
Name: target, dtype: int64


In [20]:
ScoresDFTrain.iloc[1,1:] =  scoresTrainLOF
ScoresDFTest.iloc[1,1:] =  scoresTestLOF


### 3. K nearest neighbors 

In [21]:
s = time.time()

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(xtrain,ytrain)

ypredTrainKNN = clf_knn.predict(xtrain)

s = time.time()
ypredTestKNN = clf_knn.predict(xtest)
timeKNN  = time.time()-s

print(f'Exe. time :{timeKNN}')

print(f'Accuracy on Train : {accuracy_score(ytrain,ypredTrainKNN)}')
print(f'Accuracy on Test : {accuracy_score(ytest,ypredTestKNN)}')

print(f'Recall on Train : {recall_score(ytrain,ypredTrainKNN)}')
print(f'Recall on Test : {recall_score(ytest,ypredTestKNN)}\n')

print(f'Precsion on Train : {precision_score(ytrain,ypredTrainKNN)}')
print(f'Precision on Test : {precision_score(ytest,ypredTestKNN,pos_label=1)}\n')

print(f'F1-score on Train : {f1_score(ytrain,ypredTrainKNN)}')
print(f'F1-score on Test : {f1_score(ytest,ypredTestKNN)}\n')

Exe. time :1.725247859954834
Accuracy on Train : 0.9468386023294509
Accuracy on Test : 0.6482213438735178
Recall on Train : 0.8936772046589019
Recall on Test : 0.3847980997624703

Precsion on Train : 1.0
Precision on Test : 0.2042875157629256

F1-score on Train : 0.9438537914067305
F1-score on Test : 0.2668863261943987



In [22]:
s = time.time()

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(xtrain,ytrain)

ypredTrainKNN = clf_knn.predict(xtrain)
ypredTestKNN = clf_knn.predict(xtest)



scoresTrainKNN = evalalg(ytrain,ypredTrainKNN)
scoresTestKNN = evalalg(ytest,ypredTestKNN)

In [23]:
print(scoresTrainKNN)
print(scoresTestKNN)

(0.9468386023294509, 0.8936772046589019, 1.0, 0.9438537914067305)
(0.6482213438735178, 0.3847980997624703, 0.2042875157629256, 0.2668863261943987)


In [24]:
print(pd.DataFrame(ypredTrainKNN,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestKNN,columns=['target'])['target'].value_counts())

0    6649
1    5371
Name: target, dtype: int64
0    1737
1     793
Name: target, dtype: int64


In [25]:
ScoresDFTrain.iloc[2,1:] =  scoresTrainKNN
ScoresDFTest.iloc[2,1:] =  scoresTestKNN


### 4. DBSCAN 

In [26]:

clf_dbscan_train = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtrain)
ypredTrainDBSCAN = list(clf_dbscan_train)


s = time.time()
clf_dbscan_test = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtest)

ypredTestDBSCAN = list(clf_dbscan_test)
timeDBSCAN  = time.time()-s

print(f'Exe. time :{timeDBSCAN}')


ypredTrainDBSCAN  =  redefine_pred_DBSCAN(ypredTrainDBSCAN)
ypredTestDBSCAN  =  redefine_pred_DBSCAN(ypredTestDBSCAN)

print(f'Accuracy on Train : {accuracy_score(ytrain,ypredTrainDBSCAN)}')
print(f'Accuracy on Test : {accuracy_score(ytest,ypredTestDBSCAN)}')

print(f'Recall on Train : {recall_score(ytrain,ypredTrainDBSCAN)}')
print(f'Recall on Test : {recall_score(ytest,ypredTestDBSCAN)}\n')

print(f'Precsion on Train : {precision_score(ytrain,ypredTrainDBSCAN)}')
print(f'Precision on Test : {precision_score(ytest,ypredTestDBSCAN,pos_label=1)}\n')

print(f'F1-score on Train : {f1_score(ytrain,ypredTrainDBSCAN)}')
print(f'F1-score on Test : {f1_score(ytest,ypredTestDBSCAN)}\n')

Exe. time :0.12747907638549805
Accuracy on Train : 0.5
Accuracy on Test : 0.16640316205533598
Recall on Train : 1.0
Recall on Test : 1.0

Precsion on Train : 0.5
Precision on Test : 0.16640316205533598

F1-score on Train : 0.6666666666666666
F1-score on Test : 0.2853270077939682



In [27]:
clf_dbscan_train = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtrain)

clf_dbscan_test = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtest)

ypredTrainDBSCAN = list(clf_dbscan_train)
ypredTestDBSCAN = list(clf_dbscan_test)
ypredTrainDBSCAN  =  redefine_pred_DBSCAN(ypredTrainDBSCAN)
ypredTestDBSCAN  =  redefine_pred_DBSCAN(ypredTestDBSCAN)

scoresTrainDBSCAN = evalalg(ytrain,ypredTrainDBSCAN)
scoresTestDBSCAN = evalalg(ytest,ypredTestDBSCAN)


In [28]:
print(scoresTrainDBSCAN)
print(scoresTestDBSCAN)

(0.5, 1.0, 0.5, 0.6666666666666666)
(0.16640316205533598, 1.0, 0.16640316205533598, 0.2853270077939682)


In [29]:
print(pd.DataFrame(ypredTrainDBSCAN,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestDBSCAN,columns=['target'])['target'].value_counts())

1    12020
Name: target, dtype: int64
1    2530
Name: target, dtype: int64


In [30]:
ScoresDFTrain.iloc[3,1:] =  scoresTrainDBSCAN
ScoresDFTest.iloc[3,1:] =  scoresTestDBSCAN


### 5. SVM 

In [31]:
clf_svm = svm.SVC()
clf_svm.fit(xtrain,ytrain)
ypredTrainSVM = clf_svm.predict(xtrain)
s = time.time()
ypredTestSVM = clf_svm.predict(xtest)
timeSVM  = time.time()-s

print(f'Exe. time :{timeSVM}')


print(f'Accuracy on Train : {accuracy_score(ytrain,ypredTrainSVM)}' )
print(f'Accuracy on Test : {accuracy_score(ytest,ypredTestSVM)}' )

print(f'Recall on Train : {recall_score(ytrain,ypredTrainSVM)}')
print(f'Recall on Test : {recall_score(ytest,ypredTestSVM)}\n')

print(f'Precsion on Train : {precision_score(ytrain,ypredTrainSVM)}')
print(f'Precision on Test : {precision_score(ytest,ypredTestSVM,pos_label=1)}\n')

print(f'F1-score on Train : {f1_score(ytrain,ypredTrainSVM)}')
print(f'F1-score on Test : {f1_score(ytest,ypredTestSVM)}\n')

Exe. time :4.5726470947265625
Accuracy on Train : 0.8196339434276206
Accuracy on Test : 0.44387351778656126
Recall on Train : 0.840432612312812
Recall on Test : 0.6318289786223278

Precsion on Train : 0.8068690095846646
Precision on Test : 0.17523056653491437

F1-score on Train : 0.8233088834555827
F1-score on Test : 0.2743682310469314



In [32]:
clf_svm = svm.SVC()
clf_svm.fit(xtrain,ytrain)
ypredTrainSVM = clf_svm.predict(xtrain)
ypredTestSVM = clf_svm.predict(xtest)

scoresTrainSVM = evalalg(ytrain,ypredTrainSVM)
scoresTestSVM = evalalg(ytest,ypredTestSVM)


In [33]:
print(scoresTrainSVM)
print(scoresTestSVM)

(0.8196339434276206, 0.840432612312812, 0.8068690095846646, 0.8233088834555827)
(0.44387351778656126, 0.6318289786223278, 0.17523056653491437, 0.2743682310469314)


In [34]:
print(pd.DataFrame(ypredTrainSVM,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestSVM,columns=['target'])['target'].value_counts())

1    6260
0    5760
Name: target, dtype: int64
1    1518
0    1012
Name: target, dtype: int64


In [35]:
ScoresDFTrain.iloc[4,1:] =  scoresTrainSVM
ScoresDFTest.iloc[4,1:] =  scoresTestSVM


### 6. PCA NN AD  

In [36]:
CalibParamsLoaded = load(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Results\Model\CalibParams2405_2.pkl',allow_pickle=True)
netLoaded = NeuralNetwork()
netLoaded.load_state_dict(torch.load(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Results\Model\modelCustomLoss2405_2.h5'))

<All keys matched successfully>

In [37]:
k = 20

# Learning on TS 

model = PCA(n_components=k)
model.fit(xtrain)
r_train = model.inverse_transform(model.transform(xtrain))
r_test = model.inverse_transform(model.transform(xtest))


reconsRawTrain = r_train
reconsRawTest = r_test


reconsRawTrain = torch.tensor(reconsRawTrain)
ytrain = torch.tensor(y_train.reshape(-1,1))


reconsRawTest = torch.tensor(reconsRawTest)
ytest = torch.tensor(y_test.reshape(-1,1))
xtrain = np.array(xtrain,dtype=float)
ytrain = np.array(y_train,dtype=float)

xtest = np.array(xtest,dtype=float)
ytest = np.array(y_test,dtype=float)

xtrain = torch.tensor(xtrain)
ytrain = torch.tensor(ytrain.reshape(-1,1))


xtest = torch.tensor(xtest)
ytest = torch.tensor(ytest.reshape(-1,1))

ErrRawTrain = xtrain - reconsRawTrain
ErrRawTrain = torch.tensor(ErrRawTrain)

ErrRawTest = xtest - reconsRawTest
ErrRawTest = torch.tensor(ErrRawTest)




In [38]:
anomalyScoresTrain = netLoaded(ErrRawTrain.type(torch.FloatTensor))
s = time.time()
anomalyScoresTest = netLoaded(ErrRawTest.type(torch.FloatTensor))
timePCANN  = time.time()-s

print(f'Exe. time :{timePCANN}')


Exe. time :0.0035233497619628906


In [39]:
STrain,scoresTrain  = evalPerf(netLoaded,ErrRawTrain,ytrain,'Train',CalibParamsLoaded)
ScoresPCANNTrain = [scoresTrain[0],scoresTrain[2],scoresTrain[1],scoresTrain[-1]]

STest,scoresTest  = evalPerf(netLoaded,ErrRawTest,ytest,'Test',CalibParamsLoaded)
ScoresPCANNTest = [scoresTest[0],scoresTest[2],scoresTest[1],scoresTest[-1]]


-------------------Scores on Train set-------------------
Accuracy :  0.9097
Precision :  0.9736
Recall :  0.8421
F1-score: 0.9031


-------------------Scores on Test set-------------------
Accuracy :  0.8858
Precision :  0.6126
Recall :  0.8527
F1-score: 0.7130




In [40]:
PredTrain = torch.tensor([1 if s > CalibParamsLoaded['That'] else 0 for s in STrain])
PredTest = torch.tensor([1 if s > CalibParamsLoaded['That'] else 0 for s in STest])

In [41]:
print(pd.DataFrame(PredTrain,columns=['target'])['target'].value_counts())
print(pd.DataFrame(PredTest,columns=['target'])['target'].value_counts())

0    6822
1    5198
Name: target, dtype: int64
0    1944
1     586
Name: target, dtype: int64


In [42]:
ScoresDFTrain.iloc[5,1:] =  ScoresPCANNTrain
ScoresDFTest.iloc[5,1:] =  ScoresPCANNTest


In [43]:
ScoresDFTrain

,Algo,Accuracy,Recall,Precision,F1
0,IF,0.423128,0.423128,0.423128,0.423128
1,LOF,0.599501,0.599501,0.599501,0.599501
2,KNN,0.946839,0.893677,1.0,0.943854
3,DBSCAN,0.5,1.0,0.5,0.666667
4,SVM,0.819634,0.840433,0.806869,0.823309
5,PCA NN AD,0.909651,0.842097,0.973644,0.903105


In [44]:
ScoresDFTest

,Algo,Accuracy,Recall,Precision,F1
0,IF,0.696443,0.068884,0.071605,0.070218
1,LOF,0.9,0.498812,0.833333,0.624071
2,KNN,0.648221,0.384798,0.204288,0.266886
3,DBSCAN,0.166403,1.0,0.166403,0.285327
4,SVM,0.443874,0.631829,0.175231,0.274368
5,PCA NN AD,0.885771,0.852732,0.612628,0.713009


In [45]:
[timeIF,timeLOF,timeDBSCAN,timeKNN,timeSVM,timePCANN]

[0.6915252208709717,
 0.201460599899292,
 0.12747907638549805,
 1.725247859954834,
 4.5726470947265625,
 0.0035233497619628906]